In [ ]:
import sys, time
import os
import pandas as pd
import httplib2

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
from google.oauth2 import service_account
from googleapiclient import discovery
from googleapiclient.errors import HttpError

In [ ]:
def update_values(range_name,values):
    """
    Creates the batch_update the user has access to.
    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.\n"
        """
    # pylint: disable=maybe-no-member
    
    spreadsheet_id = "ID"
    value_input_option = "USER_ENTERED"
    try:

        service = sheet_service
        
        body = {
            'values': values
        }
        result = service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=range_name,
            valueInputOption=value_input_option, body=body).execute()
        return result
    except HttpError as error:
        print(f"An error occurred: {error}")
        return error



In [ ]:
scopes = ["https://www.googleapis.com/auth/drive", 
          "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]

creds = service_account.Credentials.from_service_account_file('file.json', scopes=scopes)

In [ ]:
sheet_service = discovery.build('sheets', 'v4', credentials=creds)

In [ ]:
eventID = "384285748017"

In [ ]:
#Open a browser at the event homepage
driver = webdriver.Chrome("/Users/daniel/Documents/AddAttendeesToEventBrite/chromedriver")

In [ ]:
driver.get("https://www.eventbrite.fr/attendees-add?eid=" + str(eventID))

## ACESSS TO EVENTBRITE

In [ ]:
login = "login"##LOGIN
pwd = "password" ##PASWORD
username = driver.find_element_by_id("email")
password = driver.find_element_by_id("password")
username.send_keys(login)
password.send_keys(pwd)

In [ ]:
loginButton = driver.find_elements_by_tag_name('button')[1]
loginButton.click()
time.sleep(10)

## Sheet with Attendees Data

In [ ]:
SPREADSHEET_ID = 'ID'
DATA_TO_PULL = 'SS' ##SSName
sheet = sheet_service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=DATA_TO_PULL).execute()
data = result.get('values', [])

In [ ]:
df = pd.DataFrame(data[1:], columns=data[0])

In [ ]:
df.columns = ['surname','firstname','email','type','ticketID','EventBrite']

In [ ]:
df.shape

In [ ]:
attendees = df.T.to_dict()

In [ ]:
ticket_folder = "tickets/"
download_folder = "~/Downloads/"

In [ ]:
j=-1

In [ ]:
tempsDattent=2.5

In [ ]:
sheetName = "ssName!F"
buyerEmail = "name.surname@gmail.com"

In [ ]:
for i,att in enumerate(attendees):
    if i<=j:
        continue
    #get Attendee
    att = attendees[i]
    
    firstname = att["firstname"]
    surname = att["surname"]
    email = att["email"]
    ticketID = att["ticketID"]
    
    print(str(i)+". Adding " + firstname + " " + surname + " (" + email + ")")
    
    #Begin adding
    driver.get("https://www.eventbrite.fr/attendees-add?eid=" + str(eventID))
    
    #no payment
    select_box = driver.find_elements_by_tag_name("select")[0]
    select_box.send_keys("Gratuit")
    time.sleep(tempsDattent)
    
    #add attendee type
    quantity = driver.find_element_by_id(ticketID)
    quantity.send_keys("1")
    continueBtn = driver.find_element_by_id("continue-attendee")
    continueBtn.click()
    time.sleep(3*tempsDattent)
    
     #add name and surname
    iframe = driver.find_element_by_tag_name("iframe")
    driver.switch_to.frame(iframe)
    time.sleep(2.5*tempsDattent)
    first_name_element = driver.find_element_by_id("buyer.N-first_name")
    first_name_element.send_keys(firstname)
    last_name_element = driver.find_element_by_id("buyer.N-last_name")
    last_name_element.send_keys(surname)
    email_element = driver.find_element_by_id("buyer.N-email")
    email_element.send_keys(buyerEmail)
    sendEmailButton = driver.find_element_by_id("check-chunky_svg__eds-icon--check-chunky_svg")
    sendEmailButton.click()
    inscrireButton = driver.find_elements_by_tag_name('button')[0]
    inscrireButton.click()
    time.sleep(3*tempsDattent)
    
    #download ticket
    select_box = driver.find_elements_by_tag_name("select")[7]
    select_box.send_keys("Imprimer les billets")
    time.sleep(3*tempsDattent)
    
    #save in folder
    current_url = driver.current_url
    orderNumber = current_url.split(',')[1].split("&")[0]
    fileName =  eventID+"-"+orderNumber+"-billet.pdf"
    newFileName = firstname+"-"+surname+"-billet.pdf"
    os.rename(download_folder+fileName, ticket_folder+newFileName)
    
    ##update google sheet to values
    update_values(sheetName+str(i+2),
                  [
                      ['Oui']
                  ])
    
    j=i